In [1]:
import os
from datetime import datetime
from datetime import date
import numpy as np
import pandas as pd
# debsable warnings
import warnings
warnings.filterwarnings('ignore')

# change directory
os.chdir("../")

# print current directory
os.getcwd()

'C:\\Users\\PICHAU\\MarioCesa_Turma780_AnalisePrefixado'

In [2]:
data_feriado = pd.read_csv('../MarioCesa_Turma780_AnalisePrefixado/data/feriados_nacionais_utf8.csv', sep=';')
feriado_list = data_feriado.dropna().Data.tolist()

In [3]:
def calcula_prazo(dt_ini, dt_fim=[], feriados=[] , convencao = 'DU'):
    
    feriados_time=[]
    days_count=[]
    
    for value in feriados:
        feriados_time.append(datetime.strptime(value, '%d/%m/%Y').date())
    
    for value in dt_fim:    
        if convencao == 'DU':
            days = np.busday_count(dt_ini, value.date(), holidays=feriados_time)
            days_count.append(days)

        elif convencao == 'DC':
            days = np.busday_count(dt_ini, value.date(), weekmask= '1111111', holidays=feriados_time)
            days_count.append(days)
            
    if convencao == 'DU':
        days_div = [x / 252 for x in days_count]
        return days_count, days_div
    
    elif convencao == 'DC':
        days_div = [x / 360 for x in days_count]
        return days_count, days_div

In [4]:
def constroi_fluxo(dt_fim, frequencia='6MS'):
    date = datetime.today().date()
    
    if date.month < 7:
        date = date.replace(month=6)
        datelist = pd.date_range(date, end= dt_fim, freq= frequencia, inclusive='both')
        return datelist
    else:
        date = date.replace(month=12)
        datelist = pd.date_range(date, end= dt_fim, freq= frequencia, inclusive='both')
        return datelist

In [5]:
def calcula_pu(taxa_anual, VF=1000, prazo_anual=[]):
    
    val_cupom = []
    for value in prazo_anual:
        valor_acru = 48.808850/((1+taxa_anual)**value)
        val_cupom.append(valor_acru)
        
    valor_calc = VF/(1+taxa_anual)**prazo_anual[-1]
    val_cupom[-1] = val_cupom[-1]+valor_calc
    
    
    return val_cupom

In [6]:
lista = constroi_fluxo('01/01/2031')

In [7]:
lista

DatetimeIndex(['2022-07-01', '2023-01-01', '2023-07-01', '2024-01-01',
               '2024-07-01', '2025-01-01', '2025-07-01', '2026-01-01',
               '2026-07-01', '2027-01-01', '2027-07-01', '2028-01-01',
               '2028-07-01', '2029-01-01', '2029-07-01', '2030-01-01',
               '2030-07-01', '2031-01-01'],
              dtype='datetime64[ns]', freq='6MS')

In [8]:
dias , tn = calcula_prazo(datetime.today().date(), lista, feriados = feriado_list)

In [9]:
dias

[89,
 216,
 340,
 465,
 589,
 719,
 841,
 972,
 1094,
 1222,
 1345,
 1473,
 1597,
 1722,
 1846,
 1972,
 2095,
 2225]

In [10]:
tn

[0.3531746031746032,
 0.8571428571428571,
 1.3492063492063493,
 1.8452380952380953,
 2.3373015873015874,
 2.8531746031746033,
 3.3373015873015874,
 3.857142857142857,
 4.341269841269841,
 4.849206349206349,
 5.337301587301587,
 5.845238095238095,
 6.337301587301587,
 6.833333333333333,
 7.325396825396825,
 7.825396825396825,
 8.313492063492063,
 8.829365079365079]

In [11]:
pu = calcula_pu(0.1148, prazo_anual = tn)

In [12]:
pu

[46.971000141218,
 44.46763559751986,
 42.152189412010806,
 39.940081486061956,
 37.8603867129272,
 35.79623969508672,
 33.96159554442699,
 32.096166032032606,
 30.45115963552681,
 28.81580635197005,
 27.327138977518473,
 25.859558530928243,
 24.513041781053527,
 23.226620013420902,
 22.01720134319723,
 20.852764278747827,
 19.775479490332422,
 401.7696738647863]

In [13]:
def calcula_taxa_anual(VF, PU, prazo_anual, valor_base=100):
    
    vl_tx_anual = ((VF / PU) ** (1/prazo_anual)) - 1
    
    print(f'{vl_tx_anual*valor_base:.2f}% ao ano.')
    
    return vl_tx_anual*valor_base

In [14]:
taxa = calcula_taxa_anual(48.8088 , pu[1], tn[1])

11.48% ao ano.


In [15]:
def calcula_pu_ntnf(dt_venc, tir, feriados, dt_base=datetime.today().date()):
    
    lista_vf = []
    lista_fator_desc= []
    
    lista = constroi_fluxo(dt_venc)
    dias , tn = calcula_prazo(dt_base, lista, feriados = feriados)
    pu = calcula_pu(tir, prazo_anual = tn)
    lista = lista.strftime('%d/%m/%Y')
    
    for value in range(len(lista)):
        if value != len(lista)-1:
            lista_vf.append(48.8088)
        else:
            lista_vf.append(1048.8088)
            
    for value in tn:
        fator_desc = (1+tir)**value
        lista_fator_desc.append(fator_desc)
    
    df = pd.DataFrame(list(zip(lista, lista_vf, dias, tn, pu, lista_fator_desc )),
               columns =['Data', 'VF','Prazo Dias Uteis','Prazo Anualizado',
                         'PU', 'Fator de Desconto']).set_index('Data')
    pu_soma = sum(pu)
    
    print(f'O valor de PU é R${pu_soma:.2f}')
    
    return pu_soma, df

In [16]:
pu, df = calcula_pu_ntnf('01/01/2031', 0.1148, feriados=feriado_list )

O valor de PU é R$937.85


In [17]:
df

,VF,Prazo Dias Uteis,Prazo Anualizado,PU,Fator de Desconto
Data,,,,,
01/07/2022,48.8088,89,0.353175,46.971000,1.039127
01/01/2023,48.8088,216,0.857143,44.467636,1.097626
01/07/2023,48.8088,340,1.349206,42.152189,1.157920
01/01/2024,48.8088,465,1.845238,39.940081,1.222052
01/07/2024,48.8088,589,2.337302,37.860387,1.289180
01/01/2025,48.8088,719,2.853175,35.796240,1.363519
01/07/2025,48.8088,841,3.337302,33.961596,1.437178
01/01/2026,48.8088,972,3.857143,32.096166,1.520707
01/07/2026,48.8088,1094,4.341270,30.451160,1.602857
